In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils import data
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu116
True


In [ ]:
!pip install torchsummary
from torchsummary import summary as summary_

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Conv_block(nn.Module):
  def __init__(self, in_channels, out_channels, activation=False, **kwargs):
    super(Conv_block, self).__init__()
    self.relu = nn.ReLU()
    self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
    self.batchnorm = nn.BatchNorm2d(out_channels)
    self.activation = activation
  
  def forward(self, x):
    if not self.activation:
      return self.batchnorm(self.conv(x))
    return self.relu(self.batchnorm(self.conv(x)))

In [ ]:
class ResBlock(nn.Module):
  def __init__(self, in_channels, hidden_, out_channels) -> None:
    super(ResBlock, self).__init__()
    self.relu = nn.ReLU()

    if out_channels==64:
      self.convs = nn.Sequential(
          Conv_block(in_channels, hidden_, kernel_size=3, padding=1),
          Conv_block(hidden_, hidden_, kernel_size=3, padding=1),
          Conv_block(hidden_, out_channels, activation=False, padding=1, kernel_size=3)
      )
      self.iden = nn.Identity()
    elif in_channels == out_channels:
      self.convs = nn.Sequential(
        Conv_block(in_channels, hidden_, kernel_size=3, padding=1),
        Conv_block(hidden_, hidden_, kernel_size=3, padding=1),
        Conv_block(hidden_, out_channels, activation=False, kernel_size=3, padding=1)
      )
      self.iden = nn.Identity()
    else : 
      self.convs = nn.Sequential(
        Conv_block(in_channels, hidden_, kernel_size=3, padding=1, stride=2),
        Conv_block(hidden_, hidden_, kernel_size=3, padding=1),
        Conv_block(hidden_, out_channels, activation=False, kernel_size=3, padding=1)
      )
      self.iden = nn.Conv2d(in_channels, out_channels, kernel_size=1,stride=2)

  def forward(self, x):
    y = self.convs(x)
    x = y + self.iden(x)
    x = self.relu(x)
    return x

In [ ]:
class ResNet34(nn.Module):
    def __init__(self, in_channels=3 , num_classes=1000, is_plain=False):
        self.num_classes = num_classes
        super(ResNet34, self).__init__()
        self.conv1 = Conv_block(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.conv2_x = nn.Sequential(
                                        ResBlock(64, 64, 64),
                                        ResBlock(64, 64, 64),
                                        ResBlock(64, 64, 64)
        )
        
        self.conv3_x = nn.Sequential(
                                        ResBlock(64, 128, 128),
                                        ResBlock(128, 128, 128),
                                        ResBlock(128, 128, 128),
                                        ResBlock(128, 128, 128)
        )

        self.conv4_x = nn.Sequential(
                                        ResBlock(128, 256, 256),
                                        ResBlock(256, 256, 256),
                                        ResBlock(256, 256, 256),
                                        ResBlock(256, 256, 256),
                                        ResBlock(256, 256, 256),
                                        ResBlock(256, 256, 256)
        )
        
        self.conv5_x = nn.Sequential(
                                        ResBlock(256, 512, 512,),
                                        ResBlock(512, 512, 512),
                                        ResBlock(512, 512, 512)
        )

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc = nn.Linear(512,num_classes)

    def forward(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

In [ ]:
device = 'cuda'

In [ ]:
def build_resnet(input_shape=(3, 224, 224), **kwargs):
    x = torch.randn(2, *input_shape).to(device)
    model = ResNet34(**kwargs).to(device)
    print("ResNet34 Created")
    print(summary_(model, (3, 224, 224), batch_size=2))
    return model

In [ ]:
build_resnet()

ResNet34 Created
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [2, 64, 112, 112]           9,472
       BatchNorm2d-2          [2, 64, 112, 112]             128
        Conv_block-3          [2, 64, 112, 112]               0
         MaxPool2d-4            [2, 64, 56, 56]               0
            Conv2d-5            [2, 64, 56, 56]          36,928
       BatchNorm2d-6            [2, 64, 56, 56]             128
        Conv_block-7            [2, 64, 56, 56]               0
            Conv2d-8            [2, 64, 56, 56]          36,928
       BatchNorm2d-9            [2, 64, 56, 56]             128
       Conv_block-10            [2, 64, 56, 56]               0
           Conv2d-11            [2, 64, 56, 56]          36,928
      BatchNorm2d-12            [2, 64, 56, 56]             128
       Conv_block-13            [2, 64, 56, 56]               0
         Identity-14  

ResNet34(
  (conv1): Conv_block(
    (relu): ReLU()
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2_x): Sequential(
    (0): ResBlock(
      (relu): ReLU()
      (convs): Sequential(
        (0): Conv_block(
          (relu): ReLU()
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): Conv_block(
          (relu): ReLU()
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (2): Conv_block(
          (relu): ReLU()
          (conv): Conv2d(64, 64, kernel_size=